<font size=6><b>Параметрические критерии</b></font>

1. **Существуют две версии одновыборочного Z-критерия для доли, отличающихся формулами для статистики. Одна из этих версий более точная, поэтому мы говорили только о ней, а вторую не обсуждали вообще. Какую из этих двух версий одновыборочных Z-критериев для доли мы проходили?**
    - Версия множителей Лагранжа со статистикой Z
    - Версия Вальда со статистикой Z
    
    
_**Подсказка:**_ Ответ на этот вопрос можно найти в видео “Гипотезы о долях” на 2 уроке: https://www.coursera.org/learn/stats-for-data-analysis/lecture/rHs1E/gipotiezy-o-doliakh

2. **Z-критерий для двух долей в связанных выборках использует только информацию о**
    -  количестве несовпадающих пар значений признаков — (0,1)(0,1) и (1,0)(1,0), и порядке их появления;
    - количестве совпадающих пар значений признаков — (0,0)(0,0) и (1,1)(1,1), и объёме выборок n;
    - количестве несовпадающих пар значений признаков — (0,1)(0,1) и (1,0)(1,0), и объёме выборок n;
    - количестве единиц в каждой из выборок и их объёме n

_**Подсказка:**_ Ответ на этот вопрос можно найти в видео “Гипотезы о долях” на 2 уроке: https://www.coursera.org/learn/stats-for-data-analysis/lecture/rHs1E/gipotiezy-o-doliakh

3. **В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота: https://www.youtube.com/watch?v=mrr_UjNLbhE В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.** 

    **Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.**

    **Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.**
    
    
*Подсказки* Если у вас возникли затруднения, попробуйте пересмотреть видео “Пример: проверка гипотез о долях” на 2 уроке.

- Если все равно не получается сдать задание, ответьте себе на вопрос: “Являются ли выборки зависимыми или не зависимыми?”
- Если и в этом случае ответ не принимает, проверьте напротив правильной ли альтернативы вы проверяете нулевую гипотезу. - - Что означает, что зевота заразительна?

In [1]:
import numpy as np
import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [2]:
sample1 = [1 for i in range(10)] + [0 for i in range(24)]
sample2 = [1 for i in range(4)] + [0 for i in range(12)]
print(len(sample1), np.array(sample1).sum())
print(len(sample2), np.array(sample2).sum())

34 10
16 4


In [3]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):
    """
    Построение доверительногог интервала для разностей двух долей.
    """
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [4]:
def proportions_diff_z_stat_ind(sample1, sample2):
    """
    Формула для расчёта Z-статистики.
    """
    
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [5]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    """
    Формула для расчёта уровня значимости.
    * z_stat - та, статистика, которую оцениваем
      (это если использовать эту функцию для более широкого спектра статистик);
    * alternative - тип альтернативы.
    """
    
    
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [6]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(sample1, sample2))

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


Ноль входит в доверительный итервал, поэтому однозначный вывод о том, что одна группа отличается от другой мы не можем.

In [7]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(sample1, sample2)))

p-value: 0.745861


Для двусторонней альтернативы нулевая гипотеза не отвергается

In [8]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(sample1, sample2), 'greater'))

p-value: 0.372930


4. **Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:**

    **Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот**:

    - логистическая регрессия по признакам X1, X2, X3
    - логистическая регрессия по признакам X4,X5,X6

    **Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 нужно ввести 8)**


**Подсказка** Для ответа на это задание также необходимо понять являются ли выборки зависимыми или нет.
Обратите внимание, что логистическую регрессию нужно использовать с параметрами по умолчанию (поменялся солвер по умолчанию для логистической регрессии, когда создавалось задание, был другой. Поэтому ставим `solver='liblinear'` )



5. **В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.**

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression

In [10]:
df = pd.read_csv('banknotes.txt', sep='\t')

In [11]:
df

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [12]:
df.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'real'], dtype='object')

In [13]:
train_test_split?

In [14]:
df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]

,X1,X2,X3,X4,X5,X6
0,214.8,131.0,131.1,9.0,9.7,141.0
1,214.6,129.7,129.7,8.1,9.5,141.7
2,214.8,129.7,129.7,8.7,9.6,142.2
3,214.8,129.7,129.6,7.5,10.4,142.0
4,215.0,129.6,129.7,10.4,7.7,141.8
...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6
196,215.1,130.3,129.9,10.3,11.5,139.7
197,214.8,130.3,130.4,10.6,11.1,140.0
198,214.7,130.7,130.8,11.2,11.2,139.4


In [32]:
X_train, X_test, y_train, y_test = train_test_split(df[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']],
                                                    df['real'],
                                                    test_size=50,
                                                    random_state=1)

In [33]:
reg_1 = LogisticRegression(solver='liblinear')
reg_1.fit(X_train[['X1', 'X2', 'X3']], y_train)

reg_2 = LogisticRegression(solver='liblinear')
reg_2.fit(X_train[['X4', 'X5', 'X6']], y_train)

LogisticRegression(solver='liblinear')

In [34]:
reg_1_predictions = reg_1.predict(X_test[['X1', 'X2', 'X3']])
reg_2_predictions = reg_2.predict(X_test[['X4', 'X5', 'X6']])

In [35]:
df_test = X_test.copy()
df_test['y_true'] = y_test
df_test['reg_1_predictions'] = reg_1_predictions
df_test['reg_2_predictions'] = reg_2_predictions


In [36]:
reg_1_true_predictions = [1 if v == y_test.values[i] else 0 for (i, v) in enumerate(reg_1_predictions)]
reg_2_true_predictions = [1 if v == y_test.values[i] else 0 for (i, v) in enumerate(reg_2_predictions)]

reg_1_false_predictions = [1 if v != y_test.values[i] else 0 for (i, v) in enumerate(reg_1_predictions)]
reg_2_false_predictions = [1 if v != y_test.values[i] else 0 for (i, v) in enumerate(reg_2_predictions)]

In [37]:
sum(reg_1_true_predictions)

40

In [38]:
sum(reg_1_false_predictions)

10

In [39]:
sum(reg_2_true_predictions)

49

In [40]:
sum(reg_2_false_predictions)

1

In [41]:
# У нас связанные выборки

In [42]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [43]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [44]:
conf_interval = proportions_diff_confint_rel(reg_1_true_predictions, reg_2_true_predictions)
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      conf_interval)

95% confidence interval for a difference between proportions: [-0.300055, -0.059945]


In [45]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(reg_1_true_predictions, reg_2_true_predictions)))

p-value: 0.003297


In [46]:
round(-0.059945, 4)

-0.0599

6. **Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.** 

    **Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.**
    
    **Подсказки**. 
    - Обратите внимание каким видом критериев вы пользуетесь - одновыборочным или двухвыборочным.
    - Если все-таки возникают затруднения, попробуйте пересмотреть видео “Одновыборочные критерии Стьюдента”
    - Также подумайте, что означает формулировка что “программа работает”. Какая альтернатива лучше подходит под эту формулировку и почему?
    
    
7. **Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.**

In [50]:
 from scipy.stats import norm

In [66]:
population_mean = 525
sample_mean = 541.4
sample_size = 100
sd_population = 100

In [67]:
def get_z_score(sample_mean, population_mean, sd_population, sample_size):
    return float((sample_mean - population_mean)) / (float(sd_population) / np.sqrt(sample_size))


def get_p_value(z):
    return 1 - scipy.stats.norm.cdf(z)

In [68]:
p_value = get_p_value(get_z_score(sample_mean, population_mean,
                        sd_population, sample_size))
round(p_value, 4)

0.0505

In [69]:
sample_mean = 541.5

p_value = get_p_value(get_z_score(sample_mean, population_mean,
                        sd_population, sample_size))
round(p_value, 4)

0.0495